In [2]:
import pandas as pd
import seaborn as sns
import os, json
from tqdm import tqdm
from collections import Counter
from itertools import chain, combinations, cycle

import matplotlib.pyplot  as plt
import seaborn as sns
from skimage import io

import nltk
from subject_verb_object_extract import findSVOs, nlp

In [ ]:
def pos_index( sent, token, pos=False ):
    
    s = [ p[ int(pos) ] for p in sent ]
    
    for ind in range(len(s)):
        
        if s[ind] == token:
            return ind
    return None

In [1]:
def junctions( find_all_occ, pair, v=False ):
    
    
    pair_df = find_all_occ( pair )
    
    
    
    if len(pair_df) == 0:
        print('No sentences with', pair, 'at the same time!')
        return pair_df
    
    if v:
        print('Found', len(pair_df), 'sentences with', pair, '\n')
        print('Applying tokenizing...')
        print('Applying PoS tagging...')
        print('Finding brackets...')
        print('Slicing...')
    
    pair_df['PoS'] = pair_df['Caption'].apply(nltk.word_tokenize)
    pair_df['PoS'] = pair_df['PoS'].apply(nltk.pos_tag)
    
    
 
    
    def find_brackets( sent ):

        i = pos_index( sent, pair[0] )
        j = pos_index( sent, pair[1] )

        return i, j
    

    
    pair_df['Br'] = pair_df['PoS'].apply( find_brackets )
    
    def slicing( row ):
        
        Br = row['Br']
    
        return row['PoS'][ min(Br): max(Br)+1 ]
        
        
    
    

    pair_df['Sliced'] = pair_df[['PoS', 'Br']].apply(
        slicing,
#         lambda row: row['PoS'][ row['Br'][0]: row['Br'][1]+1 ],
        axis=1
    )
    

#     return pair_df[['PoS', 'Br']] .apply( lambda sent, br: sent[ br0 ] )
    return pair_df[['PoS', 'Br','Sliced']]


In [ ]:
sibling_identifiers = ['and', 'or', '&']
parent_identifiers = ['with']
child_identifiers = ['on', 'in']

In [ ]:
def pos_index_siblings( sent ):
    
    i = None
    for token in sibling_identifiers:
        if i == None:
            i = pos_index(sent, token)
    
    if i != None:
        if pos_index(sent[1: -1], 'NN', pos=True) != None:
            i = None
            
    return i

In [ ]:
def pos_index_parent( sent ):
    
    i = None
    for token in parent_identifiers:
        if i == None:
            i = pos_index(sent, token)
            
#     if i != None:
#         if pos_index(sent[1: -1], 'NN', pos=True) != None:
#             i = None
    
    return i

In [ ]:
def pos_index_child( sent ):
    
    i = None
    for token in child_identifiers:
        if i == None:
            i = pos_index(sent, token)
            
    if i != None:
        if pos_index(sent[1: -1], 'NN', pos=True) != None:
            i = None
    
    return i

In [ ]:
def determine_parent( fao, pair, v=False ):
    
    sliced = junctions( fao, pair, v=v )
    
    if len(sliced) == 0:
        print(f'Final vote for {pair}: Sibl')
        return 'Sibl'
    
    condition = sliced.Br.apply( lambda Br: Br[0] < Br[1] )
    
    sliced_norm = sliced[ condition ]
    sliced_reversed = sliced[ ~ condition ]
    
    
    cna = ['Sibl', 'Parent', 'Child']
    
    cnt = { k:0 for k in cna }
    
    
    for title, func in zip(cna, [pos_index_siblings, pos_index_parent, pos_index_child]):
    
        
        sliced_norm[ title ] = sliced_norm['Sliced'].apply( func )
    
        counts = len(sliced_norm[ title ].dropna())
        
        if v:
            print(f'{title}: {counts}')
        
        cnt[title] += counts
        
        
    if v:
        print('\n')
        
    for title, func in zip(cna, [pos_index_siblings, pos_index_child, pos_index_parent]):
    
        
        sliced_reversed[ title ] = sliced_reversed['Sliced'].apply( func )
    
        counts = len(sliced_reversed[ title ].dropna())
        if v:
            print(f'{title}: {counts}')
        
        cnt[title] += counts
    
    

    vote = max(cnt, key=cnt.get)
    
    # Crutch
    tol = 0.05
    if vote != 'Child':
        if abs(cnt['Sibl']-cnt['Parent']) / max(1, sum(cnt.values())) < tol:
            vote = 'Sibl'
    
    
    if v:
        print('\n')

        for k, v in cnt.items():
            print(f'{k}: {v}')
    
    print(f'Final vote for {pair}: {vote}')
    
    return vote

In [ ]:
def create_realtion_matrix( fao, label_list, v=False ):
    
    matrix = {
        label:{} for label in label_list
    }
    
  
    mapping = {
        'Child': 'Parent',
        'Parent': 'Child',
        'Is': 'Is',
        'Sibl': 'Sibl'
    }
    
    
    for i in range(len(label_list)):

        label = label_list[i]
  
        
        q = label_list[i:]
        
        for o in q:
            
            if label == o:
                
                matrix[label][o] = 'Is'
            else:
                
                vote = determine_parent( fao, (label, o), v=v )
                
                matrix[label][o] = vote
                
                matrix[o][label] = mapping[vote]
                
                
        
    
        
    return pd.DataFrame(matrix)

In [ ]:
def build_caption_tree( matrix ):
    
    def count_child( row ):
        return list(row.values).count( 'Child' )

    mat['cn'] = mat.apply(count_child , axis=1)

    sent_counts = mat['cn'].sort_values(ascending=False).unique()
    
    tree = {}
    for cn in sent_counts:
        tree[cn] = list(mat[ mat['cn'] == cn].index)
    return tree
    

In [ ]:
print('''
Imported functions for use outside:

\tbuild_caption_tree( matrix )

\tcreate_realtion_matrix( fao, label_list, v=False )

''')